In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Opción para ver todas las columnas del dataset en el notebook
pd.set_option('display.max_columns', 50)

# Práctico 04: Aprendizaje Supervisado

Para finalizar nuestro modelo, aplicaremos estrategias de sampling para dividir entre train y test y haremos crossvalidation sobre train. Realizaremos pruebas con varios clasificadores y evaluaremos los resultados con múltiples métricas. Por último calcularemos el feature importance y obtendremos conclusiones.

## Objetivo del práctico

### Train-Validation-Test
(obtener del práctico anterior)
- División del dataset en train/validation/test
- Estratificación

In [ ]:
url = "https://raw.githubusercontent.com/AgusCarchano/Mentorias-grupo1/master/data/bank-additional-full.csv"
df = pd.read_csv(url, sep=";")

In [ ]:
# Reemplazamos la columna y (target) por 1 y 0
df.y = df.y.replace('yes', 1)
df.y = df.y.replace('no', 0)

In [ ]:
#Diferenciamos los atributos del target
X = df.drop(columns='y')
y = df.y

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y)   #Dejamos un conjunto de test con el 20% de los casos 
#Dado que el dataset se encuentra desbalanceado (aprox. 11% del total de casos pertenece a la clase 1), empleamos el parámetro stratified en función del target (y)
#De este modo, la muestra seleccionada a partir de la división sería representativa para las dos clases

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, stratify=y_temp)

In [ ]:
#Pruebo sin hacer la doble division

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

### Preprocesamiento

- Tratamiento de valores nulos
- Estandarización
- Encoding de variables categóricas

In [ ]:
class SelectColumnsTransformer():
    def __init__(self, columns=None):
        self.columns = columns

    def transform(self, X, **transform_params):
        cpy_df = X[self.columns].copy()
        return cpy_df

    def fit(self, X, y=None, **fit_params):
        return self

#### Pipeline para el caso de clasificadores que requieren el escalado de variables (SVM, SGDClassifier)

In [ ]:
#Aplicamos las transformaciones previas a los conjuntos de Train y Validation

from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import IterativeImputer

variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]
mice_imputer = IterativeImputer(random_state=0, estimator=KNeighborsRegressor(),n_nearest_features=5)

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler())      
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])


train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

#### Pipeline para el caso de clasificadores que no requieren el escalado de variables

In [ ]:
#Aplicamos las transformaciones previas a los conjuntos de Train y Test

variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]


pipeline_numerico = Pipeline([
                             ('select_numeric_columns', SelectColumnsTransformer(variables_numericas))     #Para este modelo no es necesario escalar las variables numéricas
                            ])

pipeline_categorico = Pipeline ([('imputer', SimpleImputer(strategy='most_frequent', missing_values = 'unknown')),
                                   ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])


train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

### Definición de métricas

Definiremos las métricas a utilizar:
- Accuracy
- Precision
- Recall
- F1
- AUC
- PRAUC  

Además investigaremos como utilizar el classification report y confusion matrix. Adicionalmente, cómo usar crossvalidation.

### Testeo con varios modelos

Realizaremos varios tests con diversos tipos de modelos de scikit-learn:
- Logistic regression
- SVM
- Naive Bayes
- etc  
Usaremos crossvalidation y compararemos con validation y test.

### Modelos Tree Based

En esta instancia utilizaremos modelos que no pertenecen a la librería scikit-learn.  
Estos modelos son los más utilizados actualmente y han demostrado su efectividad en muchas competencias de Kaggle.  
Además, tienen la ventaja de que 
- XGBoost
- LightGBM

### Optimización de Hiperparámetros

En esta sección realizaremos varios tipos de optimización de hiperparámetros para lograr mejorar nuestras métricas.
- Grid Search
- Randomized Search

#### Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

search_params = {
    'n_estimators': [100, 500, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 6, 10, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf':[1,2,3]
}

forest_clf = RandomForestClassifier(random_state=42)
forest_clf = GridSearchCV(forest_clf, search_params, cv=5, scoring='f1', n_jobs=-1)
forest_clf.fit(train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [ ]:
display ('Best configuraton:')
display(forest_clf.best_params_)

best_forest_clf = forest_clf.best_estimator_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42,
                                    criterion = 'gini',
                                    max_depth = 15,
                                    min_samples_leaf = 1,
                                    min_samples_split = 2,
                                    n_estimators = 500)


In [ ]:
forest_clf.fit(train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= forest_clf.predict(train)
y_val_pred=forest_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

ENTRENAMIENTO
              precision    recall  f1-score   support

           0       0.93      1.00      0.97     29238
           1       0.97      0.45      0.62      3712

    accuracy                           0.94     32950
   macro avg       0.95      0.73      0.79     32950
weighted avg       0.94      0.94      0.93     32950

VALIDACIÓN
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      7310
           1       0.64      0.24      0.35       928

    accuracy                           0.90      8238
   macro avg       0.78      0.61      0.65      8238
weighted avg       0.88      0.90      0.88      8238



#### SGDClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings("ignore")

parameters = {'model__loss':['hinge', 'log','squared_loss'], 'model__learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'], 
              'model__penalty': ['l2', 'l1', 'elasticnet'], 'model__eta0': [ 1e-6,0.001, 0.01, 0.1, 1, 10]}


pipeline = Pipeline([('model',SGDClassifier( random_state= 1))])
clf = GridSearchCV(pipeline, parameters, scoring=('f1','roc_auc'), cv = 5,return_train_score=True, refit='f1')

clf.fit(train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('model',
                                        SGDClassifier(alpha=0.0001,
                                                      average=False,
                                                      class_weight=None,
                                                      early_stopping=False,
                                                      epsilon=0.1, eta0=0.0,
                                                      fit_intercept=True,
                                                      l1_ratio=0.15,
                                                      learning_rate='optimal',
                                                      loss='hinge',
                                                      max_iter=1000,
                                                      n_iter_no_change=5,
                                                      n_jobs=None, penalty

In [ ]:
display ('Best configuraton:')
display(clf.best_params_)

best_sgd_clf = clf.best_estimator_


'Best configuraton:'

{'model__eta0': 10,
 'model__learning_rate': 'constant',
 'model__loss': 'hinge',
 'model__penalty': 'l1'}

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= best_sgd_clf.predict(train)
y_val_pred=best_sgd_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

ENTRENAMIENTO
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     23390
           1       0.33      0.02      0.03      2970

    accuracy                           0.89     26360
   macro avg       0.61      0.51      0.49     26360
weighted avg       0.83      0.89      0.84     26360

VALIDACIÓN
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      5848
           1       0.31      0.02      0.03       742

    accuracy                           0.89      6590
   macro avg       0.60      0.51      0.48      6590
weighted avg       0.82      0.89      0.84      6590



#### Naive bayes

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB

In [ ]:
variables_categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
variables_numericas = ['age', 'campaign', 'previous', 'cons.conf.idx', 'euribor3m']

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             # ('standard_scaler', StandardScaler()),
                             ("kbins_discretizer", KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="uniform")),   #strategy="uniform"
                             ('bins_cat', OneHotEncoder())
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),      #podríamos no ponerlo, y que deje "desconocido" como una categoría más
                                 ('cat', OneHotEncoder())
                                 ])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                   ('cat', pipeline_categorico, variables_categoricas),
                                  ])

pipeline_modelo = Pipeline([('preprocess', pipeline_completo),
                            ('nb', ComplementNB())])

#The Complement Naive Bayes classifier was designed to correct the “severe assumptions” made by the standard Multinomial Naive Bayes classifier. It is particularly suited for imbalanced data sets.
#En el pre-procesamiento transformé todos los atributos en categóricos, porque es el requerimiento del tipo de modelo

In [ ]:
#Solo Pre-procesamiento
train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.transform(X_v)

In [ ]:
#Modelo más sencillo
nb=ComplementNB()
nb.fit(train, y_train)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, nb.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, nb.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.95      0.74      0.83     23390
           1       0.25      0.68      0.37      2970

    accuracy                           0.73     26360
   macro avg       0.60      0.71      0.60     26360
weighted avg       0.87      0.73      0.78     26360

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.95      0.75      0.84      5848
           1       0.26      0.68      0.37       742

    accuracy                           0.74      6590
   macro avg       0.60      0.72      0.61      6590
weighted avg       0.87      0.74      0.79      6590



In [ ]:
#Grilla de parámetros para optimización
params={'alpha':[1, 0.5, 0.3, 0.1, 1.2, 1.5],
        'fit_prior':[True, False],
        'norm':[True,False]
       }

nb=ComplementNB()

In [ ]:
#Búsqueda de los mejores parámetros
cv_nb = GridSearchCV(nb, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_nb.fit(train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=ComplementNB(alpha=1.0, class_prior=None, fit_prior=True,
                                    norm=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1, 0.5, 0.3, 0.1, 1.2, 1.5],
                         'fit_prior': [True, False], 'norm': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [ ]:
cv_nb.best_params_

{'alpha': 0.3, 'fit_prior': True, 'norm': True}

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo
nb_best = ComplementNB(alpha=0.3, fit_prior=True, norm=True)
nb_best.fit(train, y_train)

ComplementNB(alpha=0.3, class_prior=None, fit_prior=True, norm=True)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, nb_best.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, nb_best.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.93      0.90      0.92     23390
           1       0.37      0.44      0.40      2970

    accuracy                           0.85     26360
   macro avg       0.65      0.67      0.66     26360
weighted avg       0.86      0.85      0.86     26360

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      5848
           1       0.38      0.41      0.39       742

    accuracy                           0.86      6590
   macro avg       0.65      0.66      0.66      6590
weighted avg       0.86      0.86      0.86      6590



#### XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
variables_categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
variables_numericas = ['age', 'campaign', 'previous', 'cons.conf.idx', 'euribor3m']

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),      #podríamos no ponerlo, y que deje "desconocido" como una categoría más
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                   ('cat', pipeline_categorico, variables_categoricas),
                                  ])

pipeline_modelo = Pipeline([('preprocess', pipeline_completo),
                            ('xgb', xgb.XGBClassifier(seed=0))])

In [ ]:
#Solo Pre-procesamiento
train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.transform(X_v)

In [ ]:
xgb=xgb.XGBClassifier(seed=0)
xgb.fit(train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=None, subsample=1, verbosity=1)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     23390
           1       0.72      0.22      0.33      2970

    accuracy                           0.90     26360
   macro avg       0.81      0.60      0.64     26360
weighted avg       0.89      0.90      0.88     26360

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.90      0.99      0.95      5848
           1       0.70      0.18      0.29       742

    accuracy                           0.90      6590
   macro avg       0.80      0.58      0.62      6590
weighted avg       0.88      0.90      0.87      6590



In [ ]:
#Optimización de hiperparámetros
#Grilla de parámetros
params={'objective':["binary:logistic","binary:hinge"],
        'learning_rate':[ 0.1,0.2,0.3],
        'max_depth':[2,4, 6, 7, 8, 10],
        'alpha':[2, 3, 5, 7],
        "n_estimators":[5, 7, 10]
       }
xgb=xgb.XGBClassifier(seed=0)

In [ ]:
#Búsqueda de parámetros
cv_xgb = GridSearchCV(xgb, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_xgb.fit(train, y_train)

In [ ]:
cv_xgb.best_params_
#EN ESTA NOTEBOOK NO FUNCIONÓ, PERO EN LA OTRA ME DIO: 

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=0, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [2, 3, 5, 7],
                         'learning_rate': [0.1, 0.2, 0.3],
                         'max_depth': [2, 4, 6, 7, 8, 10],
                         'n_estimators': [5, 7, 10],
                         'objective': ['binary:logistic', 'binary:hinge']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)


{'alpha': 2,
 'learning_rate': 0.1,
 'max_depth': 7,
 'n_estimators': 7,
 'objective': 'binary:hinge'}


In [ ]:
import xgboost as xgb

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo
xgb_best = xgb.XGBClassifier(seed=0, alpha= 2, learning_rate= 0.1, max_depth= 7, n_estimators=7, objective='binary:hinge')
xgb_best.fit(train, y_train)

XGBClassifier(alpha=2, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=7, n_jobs=1,
              nthread=None, objective='binary:hinge', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=None, subsample=1, verbosity=1)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb_best.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb_best.predict(val)))

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     23390
           1       0.46      0.57      0.50      2970

    accuracy                           0.87     26360
   macro avg       0.70      0.74      0.72     26360
weighted avg       0.89      0.87      0.88     26360

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      5848
           1       0.46      0.56      0.50       742

    accuracy                           0.88      6590
   macro avg       0.70      0.74      0.72      6590
weighted avg       0.89      0.88      0.88      6590



### Explainability

Realizaremos feature importance y como opcional utilizaremos la librería SHAP para analizar las predicciones.


### Presentación

Al final del práctico, es necesario hacer 3 o 4 slides que irán incluidos en la presentación final.  
Los slides deberán contener las etapas de preprocesamiento, los modelos que utilizamos, como optimizamos los hiperparámetros, cómo validamos y qué métricas utilizamos. Por último responderemos desde el punto de vista de negocio si sirve o no sirve el modelo.

### Librerías recomendadas

Utilizaremos principalmente scikit-learn, opcionalmente xgboost y lightgbm.  
Recomiendo el siguiente material:  

- https://scikit-learn.org/stable/ -> Referencia de librería scikit-learn. Contiene casi todo lo que vamos a utilizar, pipelines, preprocesamiento y varios modelos.
- https://xgboost.readthedocs.io/en/latest/ -> Librería muy utilizada debido a que tiene muy buenos resultados. Es un tipo de algoritmo "boosting tree"
- https://lightgbm.readthedocs.io/en/latest/ -> Otra librería similar a xgboost, cada vez se usa más, debido a su facilidad de uso y buenos resultados.
- https://shap.readthedocs.io/en/latest/index.html -> Librería SHAP, para realizar explainability y analizar predicciones.
- https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py -> Ejemplo de pipelines, cross_validation y optimización de hiperparámetros

### Feature importance y explainability

In [ ]:
import xgboost as xgb
variables_numericas = ['cons.price.idx', 'cons.conf.idx', 'age','duration']
variables_categoricas = ['education', 'marital','job', 'contact', 'day_of_week']

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([
                             ('standard_scaler', StandardScaler()),
                            ])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                   ('cat', OneHotEncoder(), variables_categoricas),
                                  ])

pipeline_modelo = Pipeline([('preprocess', pipeline_completo),
                            ('xgb', xgb.XGBClassifier())])


In [ ]:
pipeline_modelo.fit(X_t, y_train)

#### Obtener los nombres de las variables

In [ ]:
# Si realizamos one hot encoding, vamos a tener el problema de que se incrementan el numero de features y necesitamos la nueva lista.
numeric_features = variables_numericas
cat_features = pipeline_modelo.named_steps['preprocess'].transformers_[1][1].get_feature_names(variables_categoricas)

#### Feature importance utilizando XGBoost

In [ ]:
onehot_columns = np.array(cat_features)
numeric_features_list = np.array(numeric_features)
numeric_features_list = np.append(numeric_features_list, onehot_columns)

In [ ]:
# Es necesario ordenar las los valores del feature importance (utilizamos argsort para tener el orden de los indices)
sorted_idx = pipeline_modelo[1].feature_importances_.argsort()
plt.barh(numeric_features_list[sorted_idx], pipeline_modelo[1].feature_importances_[sorted_idx])
plt.xlabel("Xgboost Feature Importance")
plt.show()

#### Feature importance utilizando eli5

In [ ]:
import eli5
# Utilizar eli5 nos resuelve el problema de ordenar las columnas

In [ ]:
onehot_columns = cat_features
features_list = list(numeric_features)
features_list.extend(onehot_columns)

In [ ]:
eli5.explain_weights(pipeline_modelo[1], top=50, feature_names=features_list)

#### Utilizar SHAP para obtener feature importance y expainability de las predicciones (opcional)

Como opcional, podemos utilizar la librería SHAP que nos muestra un tipo de explainability por cada predicción. Estas pueden ser agregadas para obtener un feature importance global del modelo.


https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Census%20income%20classification%20with%20XGBoost.html